## data preprocess

In [2]:
# unzip dataset
!unzip -oq -O CP936 ./C2Seg_AB.zip -d ./data/src/
!unzip -oq -O CP936 ./C2Seg_AB.zip -d ./data/src/

In [6]:
# install requirements.txt
!pip install -r ./PaddleCD/requirements.txt

In [ ]:
# data statistics
# !python ./work/data_sta.py

In [ ]:
# concat smi and sar images for training
!python ./work/data_concat.py

In [16]:
# split the dataset into train set and val set
!python ./work/data_generate.py

# copy existing files to ensure the same evaluation
# !cp ./work/AB_val.txt ./data/src/val.txt
# !cp ./work/AB_train.txt ./data/src/train.txt
# !cp ./work/BW_val.txt ./data/src/val.txt
# !cp ./work/BW_train.txt ./data/src/train.txt

## model train

In [17]:
# Configuration file path and model save path
# COF_PATH = './PaddleCD/md_config/cxup_4b_AB_10k.yml'
# MD_PATH = './best_model/cxup_4b_AB_10k.pdparams'
COF_PATH = './PaddleCD/md_config/cxup_4b_BW.yml'
MD_PATH = './best_model/cxup_4b_BW.pdparams'

In [ ]:
# 模型训练
print(COF_PATH.center(100, '*'),'\n\n')
print(MD_PATH.center(100, '*'),'\n\n')


# 与paddlers训练轮数相同，训练时长55min
!python ./PaddleCD/train.py \
       --save_dir ./data/output \
       --config $COF_PATH \
       --do_eval \
       --use_vdl \
       --iters 42000 \
       --save_interval 1600 \
       --batch_size=8 \
       --log_iters 50 \
       --num_workers 2 \
       --seed 1919810

In [ ]:
# save result
!mkdir ./best_model/
!cp ./data/output/best_model/model.pdparams $MD_PATH
!find ./data/output -name '*.log' -exec cp -t ./model_log/ "{}" +

## model val

In [ ]:
# 普通测试
!python ./PaddleCD/val.py \
       --batch_size 1 \
       --config $COF_PATH \
       --model_path $MD_PATH

In [ ]:
# 普通测试
!python ./PaddleCD/val.py \
       --batch_size 12 \
       --config $COF_PATH \
       --model_path $MD_PATH

## inference

In [ ]:
# C2Seg_AB inference
!python ./PaddleCD/predict.py \
       --config $COF_PATH \
       --model_path $MD_PATH \
       --image_path ./data/src/C2Seg_AB/test/msisar \
       --image_path2 ./data/src/C2Seg_AB/test/hsi \
       --batch_size 12 \
       --save_dir ./data/src/C2Seg_AB/result

In [ ]:
# C2Seg_BW inference
!python ./PaddleCD/predict.py \
       --config $COF_PATH \
       --model_path $MD_PATH \
       --image_path ./data/src/C2Seg_BW/test/msisar \
       --image_path2 ./data/src/C2Seg_BW/test/hsi \
       --batch_size 12 \
       --save_dir ./data/src/C2Seg_BW/result